<a href="https://colab.research.google.com/github/Syed-MuhammadTaha/AdvTensorFlowPractice/blob/main/AdvanceTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Functional API with multiple inputs (Siamese Network)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
def base_network():
  input = keras.Input(shape=(28,28,))
  x = layers.Flatten()(input)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dropout(0.1)(x)
  x = layers.Dense(128, activation='relu')(x)
  return keras.Model(inputs=input, outputs=x)

In [3]:
base = base_network()
base.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28, 28)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,504 (521.50 KB)

 Trainable params: 133,504 (521.50 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
input_1 = keras.Input(shape=(28,28,))
input_2 = keras.Input(shape=(28,28,))

vec_a = base(input_1)
vec_b = base(input_2)



In [5]:
def euclidean_distance(vectors):
  x,y = vectors
  sum_square = tf.math.reduce_sum(tf.math.square(x-y), axis=1, keepdims=True)
  return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

def euclidean_shape(shape):
  shape_a, shape_b = shape
  return (shape_a[0], 1)

In [6]:
output = layers.Lambda(euclidean_distance, output_shape=euclidean_shape)([vec_a, vec_b])

In [7]:
model = keras.Model(inputs=[input_1, input_2], outputs=output)

In [12]:
# contrastive loss is similar to the triplet loss but a supervised form where pairs of image are labelled 1 or 0 based on their similarity
model.compile(loss=contrastive_loss_w_margin(margin=1), optimizer="adam", metrics=["accuracy"])

## Custom Loss

In [10]:
# contrastive fromula: Y * D^2 + (1-Y) * max(alpha - D, 0)
# Triplet loss: max(euc_dist(A,P) - euc_dist(A,N) + alpha)
def contrastive_loss(y_true, y_pred):
  margin = 1
  square_pred = tf.math.square(y_pred)
  margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
  return ((y_true * square_pred) + (1 - y_true) * margin_square)

In [11]:
# with hyperparameter
def contrastive_loss_w_margin(margin):
  def contrastive_loss(y_true, y_pred):
    square_pred = tf.math.square(y_pred)
    margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
    return ((y_true * square_pred) + (1 - y_true) * margin_square)
  return contrastive_loss

## Custom Layers

**Lambda Layers**

In [13]:
def custom_relu(x):
  return tf.math.maximum(0.0, x)

In [15]:
model = keras.Sequential([
    layers.Flatten(input_shape=(28,28)),
    layers.Lambda(custom_relu),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
])

**Custom Layer Class**

Made up of *state* (weights) and *computation* (forward pass)

In [ ]:
class SimpleDense(layers.Layer):
  # calling layer counstructor inherited by layers.Layer class which already has w and b instance variables
  # adding units and activation instance variables to the constructor
  def __init__(self, units=32, activation=None):
    super(SimpleDense, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)
  # all the code for setting w and b tensors
  def build(self, input):
    w_init = tf.keras.initializers.GlorotNormal()
    b_init = tf.zeros_initializer()
    self.w = tf.Variable(name="kernel", initial_value = w_init(shape=(input.shape[-1], self.units), dtype="float32"), trainable=True)
    self.b = tf.Variable(name="bias", initial_value = b_init(shape=(self.units,), dtype="float32"), trainable=True)

  # computation during forward pass
  def call(self, input):
    return self.activation(tf.matmul(input, self.w) + self.b)

## Custom Models

In [ ]:
class WideandDeepModel(keras.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.hidden1 = layers.Dense(units, activation=activation)
    self.hidden2 = layers.Dense(units, activation=activation)
    self.main_output = layers.Dense(1)
    self.aux_output = layers.Dense(1)
  def call(self, inputs):
    input_A, input_B = inputs
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

### Custom Callbacks

In [ ]:
class EarlyStopping(keras.callback.CallBack):
  def __init__(self, patience=0):
    super(EarlyStopping, self).__init__()
    self.patience = patience
  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") > 0.99:
      print("Reached 99% accuracy so cancelling training!")
      self.model.stop_training = True